In [ ]:
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os

pre_file_path = os.path.join('preprocessed', 'pre-processed.csv')
pre_file_path

In [ ]:
pre_df = pd.read_csv(pre_file_path, index_col= 0)
pre_df.index.name = None
pre_df

In [ ]:
fake_path = os.path.join('size_normalized_texts', 'fake')
true_path = os.path.join('size_normalized_texts', 'true')

In [ ]:
file_names = os.listdir(true_path)
file_names

In [ ]:
content = {"Fake": [], "True": []}

for file in file_names:
    file_fake_path = os.path.join(fake_path, file)
    file_true_path = os.path.join(true_path, file)

    fake_text_file = open(file_fake_path, "r", encoding="utf8")
    true_text_file = open(file_true_path, "r", encoding="utf8")

    content["Fake"].append(fake_text_file.read())
    content["True"].append(true_text_file.read())

    fake_text_file.close()
    true_text_file.close()

In [ ]:
fake_true_df = pd.DataFrame(content)
fake_true_df

In [ ]:
#fake_true_df.to_csv("fake_vs_true.csv", index=False)

In [ ]:
fake_true_df = pd.read_csv("comparison/fake_vs_true.csv")
fake_true_df

In [ ]:
news = 4

print(f'Noticia falsa:\n{fake_true_df["Fake"][news]}')
print(f'Noticia verdadeira:\n{fake_true_df["True"][news]}')